In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [14]:
def add_rsi(data):
    df = data.copy()

    df["Gains"] = df["Variation"] * (df["Variation"] >= 0)
    df["Losses"] = df["Variation"] * (df["Variation"] < 0)
    df["Avg Gain"] = df["Gains"].rolling(14).mean()
    df["Avg Loss"] = df["Losses"].rolling(14).mean()
    for i in range(14):
        df["Avg Gain"].loc[i] = df["Gains"][0:i].mean()
        df["Avg Loss"].loc[i] = df["Losses"][0:i].mean()

    df["Avg Loss"] = -1 * df["Avg Loss"]

    df['RSI'] = 0

    #df['RSI'] = 100 - (100 / (1 + ))
    
    df['RSI'] = 100 - (100 / (1 + df['Avg Gain'] / df['Avg Loss']))

    df = df.drop(["Gains", "Losses", "Avg Gain", "Avg Loss"],axis=1)

    #df['RSI'].loc[0:14] = 100 - (100 / (1 + df['Avg Gain'].loc[0:14] / df['Avg Loss'].loc[0:14]))


    return df

In [15]:
def add_macd(data):
    df = data.copy()

    df['MACD'] = df["Close"].ewm(span=12, adjust=False).mean() - df["Close"].ewm(span=26, adjust=False).mean()

    df['MACD_SIGNAL'] = df["MACD"].ewm(span=9, adjust=False).mean()

    df['MACD_H'] = df["MACD"] - df["MACD_SIGNAL"]

    df.drop(["MACD_SIGNAL"], axis=1, inplace=True)

    return df

In [16]:
confirmation = pd.read_csv("median-confirmation-time.csv")
confirmation

,Timestamp,median-confirmation-time
0,2009-01-09 23:00:00,0.000000
1,2009-01-12 23:00:00,0.000000
2,2009-01-15 23:00:00,0.000000
3,2009-01-18 23:00:00,0.000000
4,2009-01-21 23:00:00,0.000000
...,...,...
1661,2022-09-02 00:00:00,6.300595
1662,2022-09-05 00:00:00,7.164286
1663,2022-09-08 00:00:00,7.404167
1664,2022-09-11 00:00:00,6.991071


In [17]:
time = confirmation.loc[0,"Timestamp"]
time[:-9]

'2009-01-09'

In [18]:
def confirmation_time(data):
    df = pd.DataFrame({"Date":[],"Confirmation Time":[]})
    for i in range(len(data)):
        for j in range(3):
            date = datetime.strptime(data.loc[i,"Timestamp"][:-9],"%Y-%m-%d")
            date = date + timedelta(days=j)
            #print(str(date)[:-9])
            df = df.append({"Date":str(date)[:-9],"Confirmation Time":data.loc[i,"median-confirmation-time"]},ignore_index=True)
    return df

In [19]:
transactions = pd.read_csv("n-transactions.csv")
transactions

,Timestamp,n-transactions
0,2009-01-16 23:00:00,109
1,2009-01-19 23:00:00,115
2,2009-01-22 23:00:00,86
3,2009-01-25 23:00:00,97
4,2009-01-28 23:00:00,122
...,...,...
1659,2022-09-03 00:00:00,242597
1660,2022-09-06 00:00:00,268125
1661,2022-09-09 00:00:00,290849
1662,2022-09-12 00:00:00,268971


In [20]:
def network_transactions(data):
    df = pd.DataFrame({"Date":[], "Transactions":[]})

    for i in range(len(data)):
        for j in range(3):
            date = datetime.strptime(data.loc[i,"Timestamp"][:-9],"%Y-%m-%d")
            date = date + timedelta(days=j)
            #print(str(date)[:-9])
            df = df.append({"Date":str(date)[:-9],"Transactions":data.loc[i,"n-transactions"]},ignore_index=True)
    return df

In [21]:
def miners_revenue(data):
    df = pd.DataFrame({"Date":[], "Miners Revenue":[]})

    for i in range(len(data)):
        for j in range(3):
            date = datetime.strptime(data.loc[i,"Timestamp"][:-9],"%Y-%m-%d")
            date = date + timedelta(days=j)
            df = df.append({"Date":str(date)[:-9],"Miners Revenue":data.loc[i,"miners-revenue"]},ignore_index=True)

    return df

In [22]:
def standardize_col(data,col):
    df = data.copy()
    df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df

In [23]:
data = pd.read_csv("BTC-USD_.csv")
data["Variation"] = (data["Close"] - data["Close"].shift(1)) / data["Close"].shift(1)
data.drop("Adj Close",axis=1,inplace=True)
data = add_rsi(data)
data = add_macd(data)
data = standardize_col(data,"Volume")
data = standardize_col(data,"MACD")
data = standardize_col(data,"MACD_H")
data = standardize_col(data,"RSI")
data = standardize_col(data,"Variation")
confirmation = confirmation_time(pd.read_csv("median-confirmation-time.csv"))
transactions = network_transactions(pd.read_csv("n-transactions.csv"))
miners_revenue = miners_revenue(pd.read_csv("miners-revenue.csv"))
data = pd.merge(data,confirmation,on="Date")
data = pd.merge(data,transactions,on="Date")
data = pd.merge(data,miners_revenue,on="Date")
data = standardize_col(data,"Confirmation Time")
data = standardize_col(data,"Transactions")
data = standardize_col(data,"Miners Revenue")
data.dropna(inplace=True)

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
data["Transactions"].plot.kde()
plt.show()

In [25]:
data

,Date,Open,High,Low,Close,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,-0.792147,-1.859900,-2.943160,-0.057629,-0.015462,-0.917832,-2.189543,-0.817910
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,-0.792200,0.871823,-2.943160,-0.059905,-0.018242,-0.997173,-2.189543,-0.817910
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,-0.792718,-0.690803,-1.856302,-0.062400,-0.021104,-0.997173,-2.417949,-0.847329
5,2014-09-22,399.100006,406.915985,397.130005,402.152008,-0.792842,0.163318,-1.988871,-0.063948,-0.020636,-0.997173,-2.417949,-0.847329
6,2014-09-23,402.092010,441.557007,396.196991,435.790985,-0.791785,2.111516,-1.812541,-0.062275,-0.010871,-1.072907,-2.417949,-0.847329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2022-09-04,19832.470703,19999.689453,19636.816406,19986.712891,0.478227,0.148948,-1.113287,-0.844382,-0.254845,-1.202511,-0.040819,0.203364
2926,2022-09-05,19988.789063,20031.160156,19673.046875,19812.371094,0.658018,-0.278482,-1.133325,-0.831832,-0.166540,-0.875457,-0.040819,0.203364
2927,2022-09-06,19817.724609,20155.269531,18800.171875,18837.667969,1.393318,-1.325705,-1.548973,-0.892284,-0.308652,-0.875457,0.291152,0.153032
2928,2022-09-07,18837.683594,19427.171875,18644.466797,19290.324219,0.981876,0.568952,-1.180050,-0.893278,-0.249058,-0.875457,0.291152,0.153032


In [26]:
data.to_csv("BTC-USD_SIGNALS.csv",index=False)